# ソルバを使わずに解く
ここでは多次元の常微分方程式を扱う．  
題材は有名なローレンツ方程式．  
<br>
<br>
<!-- <img src=https://latex.codecogs.com/svg.image?\begin{pmatrix}\dot{x}\\\dot{y}\\\dot{z}\end{pmatrix}&space;=&space;\begin{pmatrix}-px&plus;py\\-xz&plus;rx-y\\xy-bz\end{pmatrix}>  
<br> -->

$$
\begin{bmatrix}
\dot{x}\\
\dot{y}\\
\dot{z}\\
\end{bmatrix} = \begin{bmatrix}
\ -px+py\\
\ -xz+rx-y\\
\ xy-bz\\
\end{bmatrix}
$$

ただし  

$$
\begin{bmatrix}
\ x_0\\
\ y_0\\
\ z_0\\
\end{bmatrix} = \begin{bmatrix}
\ 0\\
\ 4\\
\ 28\\
\end{bmatrix},  \quad  p = 10,  \quad  r = 28,  \quad  b = \frac{8}{3}
$$

とする．  

微分方程式の定義

In [58]:
function lorenz(X, p, r, b)
    x, y, z = X

    dx = -p*x + p*y
    dy = -x*z + r*x -y
    dz = x*y - b*z

    [dx, dy, dz]
end

lorenz (generic function with 1 method)

ルンゲクッタ法で解く．  

In [59]:
### パラメータなど ###
Δt = 0.01  # 刻み時間
T = 50.  # 終了時間
p = 10.
r = 28.
b = 8 / 3

x₀ = [0., 4., 28.]  # 初期値

### 解を格納する配列を準備 ###
t = 0:Δt:T
x = Vector{typeof(x₀)}(undef, length(t))

### ループ ###
x[1] = x₀  # 初期値代入

for i in 1:length(t)-1
    k₁ = lorenz(x[i], p, r, b)
    k₂ = lorenz(x[i] + Δt/2*k₁, p, r, b)
    k₃ = lorenz(x[i] + Δt/2*k₂, p, r, b)
    k₄ = lorenz(x[i] + Δt*k₃, p, r, b)
    x[i+1] = x[i] + (Δt/6)*(k₁ + 2k₂ + 2k₃ + k₄)
end

### プロット
解は次のように格納されている．  
```julia
x = [[x1, y1, z1], [x2, y2, z2], ...]
```

In [60]:
x[1:3]

3-element Vector{Vector{Float64}}:
 [0.0, 4.0, 28.0]
 [0.37874472116333335, 3.961136328251045, 27.270796605474093]
 [0.7179816389461029, 3.927850615194513, 26.574667409845322]

このままではある要素の時刻歴が指定できず不便なので，結合して単一の行列する．  

In [61]:
xc = permutedims(hcat(x...))
xc[1:3, :]

3×3 Matrix{Float64}:
 0.0       4.0      28.0
 0.378745  3.96114  27.2708
 0.717982  3.92785  26.5747

プロットする

In [62]:
using Plots
plot(xc[:, 1], xc[:, 2], xc[:, 3])
savefig("sl1.png")

![pic](picture/sl1.png)

****
## 高速化

In [63]:
using BenchmarkTools

進藤裕之, 佐藤健太, "1から始める Juliaプログラミング", コロナ社, 2020を参考にやってみました．

In [64]:
# 処理はベタ書きせず，全て関数に書いて実行する

struct lorenz_param{T}
    p::T
    r::T
    b::T
end

function lorenz!(dx::T, x::T, p::lorenz_param{T}) where {T}
    dx[1] = -p.p*x[1] + p.p*x[2]
    dx[2] = -x[1]*x[3] + p.r*x[1] - x[2]
    dx[3] = x[1]*x[2] - p.b*x[3]
end

function solve_rk(x₀::Vector{U}, T::U, Δt::U, param) where U
    t = 0.:Δt:T
    x = Vector{typeof(x₀)}(undef, length(t))
    x[1] = x₀  # 初期値代入

    k₁ = zero(x₀)
    k₂ = zero(x₀)
    k₃ = zero(x₀)
    k₄ = zero(x₀)
    for i in 1:length(t)-1
        lorenz!(k₁, x[i], param)
        lorenz!(k₂, x[i] .+ (Δt/2).*k₁, param)  # ベクトルや行列の和は.+や.*を使うと早い．
        lorenz!(k₃, x[i] .+ (Δt/2).*k₂, param)
        lorenz!(k₄, x[i] .+ Δt.*k₃, param)
        x[i+1] = x[i] .+ (Δt/6).*(k₁ .+ 2 .* k₂ .+ 2 .* k₃ .+ k₄)
    end

    t, x
end

function run()
    Δt = 0.01  # 刻み時間
    T = 50.  # 終了時間

    param = lorenz_param(10., 28., 8/3)

    x₀ = [0., 4., 28]  # 初期値

    solve_rk(x₀, T, Δt, param)
end
t2, x2 = run();

In [65]:
@benchmark run()

BenchmarkTools.Trial: 5227 samples with 1 evaluation.
 Range (min … max):  709.700 μs …  18.502 ms  ┊ GC (min … max):  0.00% …  0.00%
 Time  (median):     747.500 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   949.208 μs ± 599.579 μs  ┊ GC (mean ± σ):  10.41% ± 14.82%

  █▅▄▄▃▃▃▂▂▂▁▁▁▁                                                ▁
  ██████████████████▇▇▇▇▆▆▆▆▆▇▅▄▅▄▄▄▁▇███▇▇▆▇▅▆▅▃▅▅▄▄▃▅▄▅▅▅▄▃▅▄ █
  710 μs        Histogram: log(frequency) by time       3.28 ms <

 Memory estimate: 2.18 MiB, allocs estimate: 20010.

解をプロットして確認．  

In [66]:
xc2 = permutedims(hcat(x2...))
using Plots
plot(xc2[:, 1], xc2[:, 2], xc2[:, 3])
savefig("sl2.png")

![pic](picture/sl2.png)